In [41]:
import json
import os
from typing import Dict

from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Import the Gemini wrapper
# from gemini_wrapper import default_gemini

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../notebookForAiHackathon/credentials.json"

model = ChatVertexAI(model="gemini-1.5-pro-002")  # gemini-1.5-flash

# Database file path
DATABASE_FILE = "./database.json"


# Initialize database from JSON file if it exists, otherwise create empty dict
def load_database() -> Dict[str, str]:
    if os.path.exists(DATABASE_FILE):
        try:
            with open(DATABASE_FILE, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return {}
    return {}


def save_database():
    with open(DATABASE_FILE, "w") as f:
        json.dump(database, f, indent=2)


# Initialize the database
database: Dict[str, str] = load_database()

Setting up the agent's tools

In [42]:
@tool(description="Add a value to the database using a key")
def add_to_database(key: str, value: str):
    print(f"Adding {key} to database")
    database[key] = value
    save_database()

@tool(description="Update an existing value in the database using a key")
def update_database(key: str, value: str):
    print(f"Updating {key}: {value}")
    # TODO: Need guardrails when updating, checking if key exists, etc
    database[key] = value
    save_database()

@tool(description="retrieve a value from the database using a key")
def get_from_database(key: str) -> str:
    print(f"Getting {key} from database")
    return database.get(key, "Key not found")

@tool(description="Get all the keys from the database")
def get_all_keys_from_database() -> list[str]:
    print("Getting all keys from the database")
    return list(database.keys())

@tool(description="Delete a value from the database using a key")
def delete_from_database(key: str):
    print(f"Deleting {key} from database")
    if key in database:
        del database[key]
        save_database()

@tool(description="Clear all the keys and values from the database")
def clear_database():
    print("Clearing all keys and values from the database")
    database.clear()
    save_database()

In [43]:
def create_database_agent():

    # Create the list of tools
    tools = [
        add_to_database,
        get_from_database,
        get_all_keys_from_database,
        delete_from_database,
        update_database
    ]

    # Initialize the model using the Gemini wrapper
    model_with_tools = model.bind_tools(tools)

    # Create the prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a helpful assistant named Amy with access to a database.
        You can:
        - Add key-value pairs to the database
        - Retrieve values by key
        - List all keys in the database
        - Delete specific key-value pairs
        - Clear the entire database

        Always use the tools to interact with the database, don't rely on your internal memory.

        Always infer the best key to use when creating new data, use camel case. First, list the keys to see if the key is available before inferring.

        Always be polite and helpful when interacting with the database.""",
            ),
            ("placeholder", "{messages}"),
        ]
    )

    # Create memory and agent
    memory = MemorySaver()
    agent_executor = create_react_agent(
        model_with_tools, tools, prompt=prompt, checkpointer=memory
    )

    return agent_executor


def invoke(utterance: str) -> str:
    """Invoke the database agent with a user utterance."""
    print(f"> {utterance}")
    thread_id = "db_agent_123"
    response = agent_executor.invoke(
        {"messages": [HumanMessage(content=utterance)]},
        config={"configurable": {"thread_id": thread_id}},
    )
    messages = response["messages"]
    ai_message = messages[-1].content
    return ai_message


def chat():
    """Interactive chat loop for the database agent."""
    while True:
        utterance = input(">").strip()
        if "q" == utterance.casefold():
            break
        print(invoke(utterance))
    print("Goodbye")


In [44]:
if __name__ == "__main__":
    # Create the agent
    agent_executor = create_database_agent()

    # Start the chat loop
    print(
        "Database Agent: Hello! I can help you manage the database. What would you like to do?"
    )
    chat()

Database Agent: Hello! I can help you manage the database. What would you like to do?
> How many legs does my dog have?
Please tell me your dog's name and how many legs it has. I can then store that information in my database.

> My dogs name is frank, and frank has a normal amount of legs
Getting all keys from the database
Adding dogFrankLegs to database
Frank has 4 legs. I have stored this information in my database under the key dogFrankLegs.



KeyboardInterrupt: Interrupted by user